In [5]:
import glob
from os.path import dirname, basename
import pandas as pd
from inpainting.load import load_dataframe


In [6]:
data_dir = '../data'
results_dirs = glob.glob('../results/image_inpainting/*')


object_stats_dfs = []
for path in glob.glob(f'{data_dir}/interim/DAVIS/ObjectStats/*/raw_object_stats.csv'):
    object_stats_df = load_dataframe(path)
    object_stats_df['object_sequence_name'] = basename(dirname(path))
    object_stats_dfs.append(object_stats_df)
object_stats_df = pd.concat(object_stats_dfs)

times_dfs = []
for results_dir in results_dirs:
    for path in glob.glob(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_dfs = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in glob.glob(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(object_stats_df, results_df, on=['object_sequence_name', 'object_id', 'frame_id'])
df = pd.merge(df, times_dfs, on=['model', 'sequence_name', 'frame_id'])
df

,frame_id,object_id,object_to_screen_ratio,object_sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,model,sequence_name,inference_time
0,0,1,0.077911,bike-packing,125.198766,27.154803,0.949514,ii_deepfillv1,dog-agility_bike-packing_001,37.804382
1,0,1,0.077911,bike-packing,67.449176,29.841037,0.952569,ii_deepfillv1,soapbox_bike-packing_001,39.715874
2,0,1,0.077911,bike-packing,113.169161,27.593523,0.942752,ii_deepfillv2,dog-agility_bike-packing_001,27.801727
3,0,1,0.077911,bike-packing,40.873141,32.016423,0.959284,ii_deepfillv2,soapbox_bike-packing_001,27.605888
4,0,1,0.077911,bike-packing,124.304563,27.185933,0.951844,ii_kernelfill,dog-agility_bike-packing_001,200.926407
...,...,...,...,...,...,...,...,...,...,...
28700,51,3,0.021820,walking,5.373411,40.828303,0.992048,ii_deepfillv1,drift-chicane_walking_003,45.212608
28701,51,3,0.021820,walking,7.779963,39.221028,0.991796,ii_deepfillv2,drift-chicane_walking_003,33.836929
28702,51,3,0.021820,walking,41.313344,31.969900,0.988283,ii_kernelfill,drift-chicane_walking_003,118.990845
28703,51,3,0.021820,walking,67.133268,29.861426,0.982328,ii_pconvunet,drift-chicane_walking_003,25.467968


In [7]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['inference_time'] = df['inference_time']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,inference_time,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,,
ii_deepfillv1,47.169212,314.586824,31.048661,0.938230
ii_deepfillv2,32.516945,292.230858,30.486961,0.939222
ii_kernelfill,268.042337,280.903865,33.261293,0.948482
ii_pconvunet,25.007221,471.737222,27.110769,0.926932
ii_regionfill,122.415958,250.708970,34.563039,0.950062


In [8]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['inference_time'] = df['inference_time']
metrics['mean_squared_error'] = df['mean_squared_error'] * df['object_to_screen_ratio']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio'] * df['object_to_screen_ratio']
metrics['structural_similarity'] = df['structural_similarity'] * df['object_to_screen_ratio']
metrics.groupby('model').mean()


,inference_time,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,,
ii_deepfillv1,47.169212,22.200860,1.592903,0.055602
ii_deepfillv2,32.516945,18.005398,1.632262,0.056027
ii_kernelfill,268.042337,18.620494,1.701030,0.056698
ii_pconvunet,25.007221,44.615440,1.367868,0.054479
ii_regionfill,122.415958,14.108430,1.796830,0.056896
